# 0. Imports

In [1]:
import sys
import joblib

sys.path.insert(0, "../src/")

from sklearn.pipeline import Pipeline
from feature_engine.imputation import MeanMedianImputer
from feature_engine.wrappers import SklearnTransformerWrapper
from feature_engine.discretisation import (
    EqualFrequencyDiscretiser,
    EqualWidthDiscretiser,
)
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.linear_model import LogisticRegression

from utils.utils import load_config_file
from data.data_load import DataLoad
from data.data_validation import DataValidation
from data.data_transformation import DataTransformation
from data.data_preprocess import DataPreprocess
from train.train import TrainModels
from evaluation.classifier_eval import ModelEvaluation

# 1. Data Load

In [2]:
dl = DataLoad()
df = dl.load_data("train_dataset_name")

2025-05-19 20:58:52 [info     ] Começando a carga dos dados com o nome: train_dataset_name


# 2. Data Validation

In [3]:
dv = DataValidation()
is_valid = dv.run(df)

2025-05-19 20:58:56 [info     ] Validacao iniciou             
2025-05-19 20:58:56 [info     ] Validation columns passed...  
2025-05-19 20:58:56 [info     ] Validacao com sucesso.        


# 3. Data Transformation

In [4]:
dt = DataTransformation(df)
X_train, X_valid, y_train, y_valid = dt.train_test_spliting()

# 4. Experimentations 

In [5]:
import mlflow
from mlflow.tracking import MlflowClient

In [6]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("prob_loan")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1747133663728, experiment_id='1', last_update_time=1747133663728, lifecycle_stage='active', name='prob_loan', tags={}>

## 4.1. Hyperparameters

In [7]:
from mlflow.models import MetricThreshold
from sklearn.dummy import DummyClassifier
from mlflow.models import infer_signature

In [8]:
from hyperopt import fmin, tpe, hp, STATUS_OK

In [9]:
pipe = Pipeline(
    [
        (
            "imputer",
            MeanMedianImputer(variables=["RendaMensal", "NumeroDeDependentes"]),
        ),
        (
            "discretizer",
            EqualFrequencyDiscretiser(
                variables=[
                    "TaxaDeUtilizacaoDeLinhasNaoGarantidas",
                    "TaxaDeEndividamento",
                    "RendaMensal",
                ]
            ),
        ),
        ("scaler", SklearnTransformerWrapper(StandardScaler())),
    ]
)

In [10]:
def objective(params):
    with mlflow.start_run(run_name="with_discretizer_hyperopt"):
        mlflow.set_tag("model_name", "lr_discretizer_hyperopt")
        mlflow.log_params(params)

        preprocessador = DataPreprocess(pipe)
        preprocessador.train(X_train)

        X_train_processed = preprocessador.transform(X_train)
        X_valid_processed = preprocessador.transform(X_valid)
        joblib.dump(preprocessador, "../models/preprocess.joblib")

        ## 1.1. logar um artefato (preprocessador)
        mlflow.log_artifact("../models/preprocess.joblib")

        ## 1.2. logar os parametros do preprocessador
        mlflow.log_params(
            params={
                "imputer": pipe["imputer"],
                "discretizer": pipe["discretizer"],
                "scaler": pipe["scaler"],
            }
        )

        # 2. Inicia o exprimento com cross validation
        model = LogisticRegression(**params)
        model_eval = ModelEvaluation(model, X_train_processed, y_train, n_splits=5)
        roc_auc_scores = model_eval.cross_val_evaluate()

        ## 2.1. logar o resultado da performance
        mlflow.log_metric("train_roc_auc", roc_auc_scores.mean())

        # 3. Treinar o modelo
        model.fit(X_train_processed, y_train)

        # 4. Salvar as metricas de validacao do modelo
        y_val_preds = model_eval.model.predict_proba(X_valid_processed)[:, 1]
        val_roc_auc = model_eval.evaluate_predictions(y_valid, y_val_preds)

        mlflow.log_metric("valid_roc_auc", val_roc_auc)

        # 5. Logar o modelo
        candidate_model_uri = mlflow.sklearn.log_model(model, "lr_model").model_uri

        # ===============================================================================
        signature = infer_signature(X_valid_processed, y_valid)

        eval_data = X_valid_processed
        eval_data["label"] = y_valid

        thresholds = {
            "accuracy_score": MetricThreshold(
                threshold=0.1,  # o score da acuracia precisar ser > 0.7 para modelo ser logado
                min_absolute_change=0.05,
                min_relative_change=0.05,
                greater_is_better=True,
            )
        }

        baseline_model = DummyClassifier(strategy="uniform").fit(
            X_train_processed, y_train
        )
        baseline_model_uri = mlflow.sklearn.log_model(
            baseline_model, "baseline_model", signature=signature
        ).model_uri

        # processo responsavel por avaliar o modelo do mlflow
        mlflow.evaluate(
            candidate_model_uri,
            eval_data,
            targets="label",
            model_type="classifier",
            validation_thresholds=thresholds,
            baseline_model=baseline_model_uri,
        )

        mlflow.end_run()

        return {"loss": -roc_auc_scores.mean(), "status": STATUS_OK}

In [11]:
search_space = {
    "warm_start": hp.choice("warm_start", [True, False]),
    "fit_intercept": hp.choice("fit_intercept", [True, False]),
    "tol": hp.uniform("tol", 0.00001, 0.0001),
    "C": hp.uniform("C", 0.05, 3),
    "solver": hp.choice("solver", ["newton-cg", "lbfgs", "liblinear"]),
    "max_iter": hp.choice("max_iter", range(100, 1000)),
    "multi_class": "auto",
    "class_weight": hp.choice("class_weight", [None, "balanced"]),
}

In [12]:
best_result = fmin(fn=objective, space=search_space, algo=tpe.suggest, max_evals=5)

2025-05-19 20:59:08 [info     ] Preprocessamento iniciou...   
2025-05-19 20:59:09 [info     ] Transformação dos dados com preprocessador iniciou...
2025-05-19 20:59:09 [info     ] Transformação dos dados com preprocessador iniciou...
2025-05-19 20:59:09 [info     ] Iniciou a validação cruzada...
  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

/home/felipe/miniconda3/envs/mlflow_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(

/home/felipe/miniconda3/envs/mlflow_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(

/home/felipe/miniconda3/envs/mlflow_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(

/home/felipe/miniconda3/envs/mlflow_env/lib/python3.10/

2025-05-19 20:59:11 [info     ] Iniciou a validação do modelo 
  0%|          | 0/5 [00:03<?, ?trial/s, best loss=?]

2025/05/19 20:59:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

/home/felipe/miniconda3/envs/mlflow_env/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(

/home/felipe/minicond

🏃 View run with_discretizer_hyperopt at: http://127.0.0.1:5000/#/experiments/1/runs/ddfebcee207d41689c408817173e103e

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1

 20%|██        | 1/5 [00:18<01:13, 18.28s/trial, best loss: -0.8245984820451715]

...

/home/felipe/miniconda3/envs/mlflow_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(

/home/felipe/miniconda3/envs/mlflow_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(

/home/felipe/miniconda3/envs/mlflow_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(

/home/felipe/miniconda3/envs/mlflow_env/lib/python3.10/

2025-05-19 21:00:14 [info     ] Iniciou a validação do modelo 
 80%|████████  | 4/5 [01:06<00:15, 15.26s/trial, best loss: -0.8473541789127749]

2025/05/19 21:00:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

/home/felipe/miniconda3/envs/mlflow_env/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(

/home/felipe/minicond

🏃 View run with_discretizer_hyperopt at: http://127.0.0.1:5000/#/experiments/1/runs/d3f0332c474948b89f065e2ba588fcd7

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

100%|██████████| 5/5 [01:18<00:00, 15.77s/trial, best loss: -0.8473541789127749]


<Figure size 1050x700 with 0 Axes>